In [1]:
(load "../../common")
(cl-common:hello)

(load "on-lisp")

(in-package :on-lisp)
(hello)

T

"Hello, Common!"

T

#<PACKAGE "ON-LISP">

"Hello, On Lisp!"


"Hello, Common!" 
"Hello, On Lisp!" 

# Defining Functions

In [2]:
(defun double (x)
  (* x 2))

DOUBLE

In [3]:
(double 1)

2

In [4]:
#'double

#<FUNCTION DOUBLE>

In [5]:
(function double)

#<FUNCTION DOUBLE>

In [6]:
(eq #'double (function double))

T

In [7]:
(eq #'double (car (list #'double)))

T

In [12]:
; lambda expression
; #'(lambda (x) (* x 2))
(format t "~A~%" #'(lambda (x) (* x 2)))

NIL

#<FUNCTION (LAMBDA (X)
  :IN
  "C:/Users/zhouj/AppData/Local/Temp/7EC2E24C70CC4E8CCF0FE027F8372D99-542064446.lisp") {1000D8E6CB}>


In [ ]:
(double 3)
((lambda (x) (* x 2)) 3) ; appear first in function call

6

6

In [14]:
; same function name and variable name
(let ((double 2))
  (double double))

4

In [17]:
; symbol-value
; symbol-function
(setq double 2)
(symbol-value 'double)
(symbol-function 'double)

2

2

#<FUNCTION DOUBLE>

SIMPLE-WARNING: undefined variable: ON-LISP::DOUBLE


In [18]:
; variable hold a function as its value
(setq x #'append)
(eq (symbol-value 'x) (symbol-function 'append))

#<FUNCTION APPEND>

T

SIMPLE-WARNING: undefined variable: ON-LISP::X


In [2]:
(double% 2)

4

# Functional Arguments

In [3]:
; apply, funcall
(+ 1 2)

(apply #'+ '(1 2))
(apply #'+ 1 '(2))
(apply (symbol-function '+) '(1 2))
(apply #'(lambda (x y) (+ x y)) '(1 2))

(funcall #'+ 1 2)

3

3

3

3

3

3

In [5]:
(mapcar #'(lambda (x) (+ x 10))
  '(1 2 3))
(mapcar #'+
    '(1 2 3)
  '(10 100 1000))

(sort '(1 4 2 5 6 7 3) #'<)
(remove-if #'evenp '(1 2 3 4 5 6 7))

(11 12 13)

(11 102 1003)

(1 2 3 4 5 6 7)

(1 3 5 7)

SB-INT:CONSTANT-MODIFIED: Destructive function SORT called on constant data: (1 4 2 5 6 7 3)
See also:
  The ANSI Standard, Special Operator QUOTE
  The ANSI Standard, Section 3.7.1


# Functions as Properties

In [1]:
(defun wag-tail () (princ 'wag-tail))
(defun bark () (princ 'bark))
(defun scurry () (princ 'scurry))
(defun squeak () (princ 'squeak))
(defun rub-legs () (princ 'rub-legs))
(defun scratch-carpet () (princ 'scratch-carpet))

(defun behave% (animal)
  (case animal
    (dog (wag-tail)
         (bark))
    (rat (scurry)
         (squeak))
    (cat (rub-legs)
         (scratch-carpet))))

(defun behave (animal)
  (funcall (get animal 'behavior)))

(setf (get 'dog 'behavior)
  #'(lambda ()
      (wag-tail)
      (bark)))

WAG-TAIL

BARK

SCURRY

SQUEAK

RUB-LEGS

SCRATCH-CARPET

BEHAVE%

BEHAVE

#<FUNCTION (LAMBDA ()
             :IN
             "C:/Users/zhouj/AppData/Local/Temp/E8CA5AFF4C0E46439C21409DEB4C8656-2676620219.lisp") {1000FD1A3B}>

In [3]:
(behave% 'dog)


BARK

WAG-TAILBARK

In [4]:
(behave 'dog)

BARK

WAG-TAILBARK

# Scope

In [ ]:
(let ((y 7))
  (defun scope-test (x)
    (list x y)))

(let ((y 5))
  (scope-test 3))

; (3 7)
; lexically scoped

SCOPE-TEST

(3 7)

SB-INT:SIMPLE-STYLE-WARNING: The variable Y is defined but never used.


# Closures

In [6]:
(defun list+ (lst n)
  (mapcar #'(lambda (x) (+ x n)) lst))

(list+ '(1 2 3) 10)

LIST+

(11 12 13)

In [7]:
(let ((counter 0))
  (defun new-id () (incf counter))
  (defun reset-id () (setq counter 0)))

RESET-ID

In [8]:
(defun make-adder (n)
  #'(lambda (x) (+ x n)))

(setq add2 (make-adder 2)
  add10 (make-adder 10))

(funcall add2 5)
(funcall add10 3)

MAKE-ADDER

#<FUNCTION (LAMBDA (X) :IN MAKE-ADDER) {1101F68D8B}>

7

13

SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::ADD10
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::ADD2
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::ADD2
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::ADD10


In [1]:
; change internal state
(defun make-adderb (n)
  #'(lambda (x &optional change)
      (if change
          (setq n x)
          (+ x n))))

(let ((addx (make-adderb 1)))
  (list
   (funcall addx 3)
   (funcall addx 100 t)
   (funcall addx 3)))

MAKE-ADDERB

(4 100 103)

In [ ]:
; closures share same object
(defun make-dbms (db)
  (list
   #'(lambda (key)
       (cdr (assoc key db)))
   #'(lambda (key val)
       (push (cons key val) db)
       key)
   #'(lambda (key)
       (setf db (delete key db :key #'car))
       key)))

(defun lookup (key db)
  (funcall (car db) key))

(let ((cities (make-dbms '((boston . us) (paris . france)))))
  (list
   (funcall (car cities) 'boston)
   (funcall (second cities) 'lodon 'england)
   (funcall (car cities) 'lodon)
   (lookup 'paris cities)))

MAKE-DBMS

LOOKUP

(US LODON ENGLAND FRANCE)

# Local Functions